# ICC305 Avaliação de Desempenho - 2023/1

## Atividade 3

**Giovanna** Andrade Santos - 22052563

**Marcos** Avner Pimenta de Lima - 21852448

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import math

from scipy.stats import chisquare
from scipy.stats import chi2_contingency

from ad_library import myfunctions
from collections import defaultdict

In [3]:
#nº de clientes
n1 = 10 ** 3
n2 = 10 ** 6
n3 = 10 ** 9
n4 = 10 ** 12

#taxa de entrada --> 9 clientes/s
tc = 9

#taxa de serviço --> 10 clientes/s
ts = 10

# https://introcs.cs.princeton.edu/python/43stack/mm1queue.py

In [4]:
# Accept float command-line arguments lamb and mu. Simulate an
# M/M/1 queue with arrival rate lamb and service rate mu.
def mm1(n=1000, tc=9., ts=10., gerador=myfunctions.G5RandomGenerator()):
    lamb = 1. / tc
    mu = 1. / ts
    fila = list() # fila de espera
    espera = np.zeros((1,n), dtype=int)
    cont = 0
    
    prox_chegada = myfunctions.va_exp(lamb, gerador) # computa o tempo da proxima chegada.
    prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador) # computa o tempo da conclusao do proximo atendimento.
    
    # Simulate the M/M/1 queue.
    while n > 0:

        while prox_chegada < prox_atend:
            # simula a proxima chegada
            fila.append(prox_chegada)
            prox_chegada += myfunctions.va_exp(lamb, gerador)

        # Next event is a service completion.
        if fila:
            chegada = fila.pop(0)
            espera[0][cont] = int((prox_atend - chegada)*1000)
            cont += 1

        # atualiza a fila de espera.
        if fila:
            # se a fila não estiver vazia, o termino do proximo atendimento 
            # é incrementando com o ts
            prox_atend += myfunctions.va_exp(mu, gerador)
        else:
            # se a fila estiver vazia, o termino do próximo atendimento
            # é incrementando com o (tc + ts)
            prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador)
        
        n -= 1
    return espera[0]

In [5]:
#função para calcular intervalo de confiança
def ic(n, media, values: list, alpha=.05):
    std = np.std(values)
    H = st.t.ppf(q=1. - alpha / 2., df=n-1) * std
    li = media - H
    ls = media + H
    return (li, ls)

In [6]:
g5 = myfunctions.G5RandomGenerator()
g5.reset()

for n in [n1, n2, n3]:    
    tot_espera = mm1(n, tc, ts, g5)
    soma = np.sum(tot_espera)
    media = myfunctions.mean(tot_espera)
    inter_conf = ic(n, media, tot_espera)
    print(f'N: {n}')
    print(f'Espera total: {soma}')
    print(f'Espera Média: {media}')
    print(f'Espera IC: {inter_conf}')
    print()

N: 1000
Espera total: 673838
Espera Média: 673.838
Espera IC: (-396.5929554419736, 1744.2689554419735)

N: 1000000
Espera total: 1003611329
Espera Média: 1003.611329
Espera IC: (-961.434759208849, 2968.657417208849)



KeyboardInterrupt: 

In [30]:
valor_esperado = myfunctions.valor_esperado(tc, ts)
print(f'O valor esperado do tempo de espera para os valores de lambda e mi é {valor_esperado}')

O valor esperado do tempo de espera para os valores de lambda e mi é 0.9000000000000002


In [35]:
alpha = .05


Intervalo de confiança é (-0.3966741950472672, 1.74532996122303)
